In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory():
    return memory.load_memory_variables({})["history"]

chain = prompt | llm

def invoke_chain(question):
    result = chain.invoke(
        {
            "question": question,
            "history": load_memory(),
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


In [2]:
invoke_chain("My name is Ben.")

content='Hello Ben! How can I assist you today?'


In [3]:
invoke_chain("What is my name?")

content='Your name is Ben.'
